In [1]:
import pandas as pd
from amb_sdk.sdk import DarwinSdk

In [2]:
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')

(True, 'https://amb-demo-api.sparkcognition.com/v1/')

In [3]:
#Set your user id and password accordingly
USER='idunlap@rocketmail.com'
PW='5uVGHsTHrQ'

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
PATH_TO_DATASET='sets/'
TRAIN_DATASET='17_18_Season_Stats_Per_100_Possessions_SPP.csv'
TEST_DATASET= '16_17_Season_Stats_Per_100_Possessions_SPP.csv'
# A timestamp is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

In [4]:
# Import necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report


In [5]:
status, msg = ds.auth_login_user(USER,PW)
if not status:
    print(msg)

In [6]:
# Preview dataset
df = pd.read_csv(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,string_field_29,ORtg,DRtg
0,8,Jarrett Allen\allenja01,C,19,BRK,72,31,1441,7.9,13.4,...,13.1,1.7,0.9,3.0,2.8,5.0,19.8,NaN,119.0,108
1,15,Ike Anigbogu\anigbik01,C,19,IND,11,0,30,6.7,15.0,...,15.0,0.0,1.7,5.0,3.3,1.7,21.7,NaN,114.0,104
2,4,Bam Adebayo\adebaba01,C,20,MIA,69,19,1368,6.4,12.5,...,14.0,3.7,1.2,1.5,2.4,5.1,17.5,NaN,116.0,105
3,62,Tony Bradley\bradlto01,C,20,UTA,9,0,29,5.2,19.0,...,19.0,1.7,0.0,0.0,0.0,1.7,13.8,NaN,95.0,107
4,75,Thomas Bryant\bryanth01,C,20,LAL,15,0,72,5.3,14.0,...,11.3,4.0,0.7,1.3,1.3,4.0,14.6,NaN,100.0,108


In [7]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
if not status:
    print(dataset)

400: BAD REQUEST - {"message": "Dataset already exists"}



In [8]:
# clean dataset
target = "Pos"
status, job_id = ds.clean_data(TRAIN_DATASET, target = target)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Taken', 'starttime': '2019-04-17T22:04:15.983886', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['29de94d76261471aab0fac9b57ed77fd'], 'model_name': None, 'job_error': None}
{'status': 'Complete', 'starttime': '2019-04-17T22:04:15.983886', 'endtime': '2019-04-17T22:04:18.70444', 'percent_complete': 100, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['29de94d76261471aab0fac9b57ed77fd'], 'model_name': None, 'job_error': ''}


In [9]:
model = target + "_model0" + ts
status, job_id = ds.create_model(dataset_names = TRAIN_DATASET, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Taken', 'starttime': '2019-04-17T22:04:32.199036', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'Pos_model020190417220414', 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-17T22:04:32.199036', 'endtime': None, 'percent_complete': 4, 'job_type': 'TrainModel', 'loss': 1.311550259590149, 'generations': 4, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'Pos_model020190417220414', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T22:04:32.199036', 'endtime': None, 'percent_complete': 4, 'job_type': 'TrainModel', 'loss': 1.311550259590149, 'generations': 4, 'dataset_names': ['17_18_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'Pos_model020190417220414', 'job_error': ''}


KeyboardInterrupt: 

In [ ]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

In [ ]:
feature_importance[:10]

In [ ]:
status, artifact = ds.run_model(TRAIN_DATASET, model)
sleep(1)
ds.wait_for_job(artifact['job_name'])

In [ ]:
status, prediction = ds.download_artifact(artifact['artifact_name'])
prediction.head()

In [ ]:
unq = prediction[target].unique()[::-1]
p = np.zeros((len(prediction),))
a = np.zeros((len(prediction),))
for i,q in enumerate(unq):
    p += i*(prediction[target] == q).values
    a += i*(df[target] == q).values
#Plot predictions vs actual
plt.plot(a)
plt.plot(p)
plt.legend(['Actual','Predicted'])
plt.yticks([i for i in range(len(unq))],[q for q in unq]);
print(classification_report(df[target], prediction[target]))

In [ ]:
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, TEST_DATASET))
if not status:
    print(dataset)

In [ ]:
# clean test dataset
status, job_id = ds.clean_data(TEST_DATASET, target = target, model_name = model)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
status, artifact = ds.run_model(TEST_DATASET, model)
sleep(1)
ds.wait_for_job(artifact['job_name'])

In [ ]:
# Create plots comparing predictions with actual target
status, prediction = ds.download_artifact(artifact['artifact_name'])
df = pd.read_csv(os.path.join(PATH_TO_DATASET, TEST_DATASET))
unq = prediction[target].unique()[::-1]
p = np.zeros((len(prediction),))
a = np.zeros((len(prediction),))
for i,q in enumerate(unq):
    p += i*(prediction[target] == q).values
    a += i*(df[target] == q).values
#Plot predictions vs actual
plt.plot(a)
plt.plot(p)
plt.legend(['Actual','Predicted'])
plt.yticks([i for i in range(len(unq))],[q for q in unq]);
print(classification_report(df[target], prediction[target]))

In [ ]:
status, model_type = ds.lookup_model_name(model)
print(model_type['description']['best_genome'])